# Быстрый запуск трекинга MLflow
### Цель данного ноутбука - предоставить краткое руководство по наиболее важным базовым API отслеживания MLflow. В частности, по тем, которые позволяют вести журнал, регистрировать и загружать модель для вывода

### - Как регистрировать параметры, метрики и модель

### - Основы MLflow fluent API

### - Как зарегистрировать модель во время ведения журнала

### - Как перейти к модели в пользовательском интерфейсе MLflow

### - Как загрузить зарегистрированную модель для вывода

### Первым шагом является установка пакета MLFlow
```Bash
# установить последний релиз-кандидат 
pip install mlflow 

# или установите конкретную версию rc 
pip install mlflow==2.14.0rc0

```

In [1]:
# Загрузка библиотек для работы с ноутбуком
import os
import warnings
import sys
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from itertools import cycle
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import lasso_path, enet_path
from sklearn import datasets


In [2]:
from minio import Minio
from minio.error import InvalidResponseError, S3Error

accessID = os.environ.get('AWS_ACCESS_KEY_ID')
accessSecret =  os.environ.get('AWS_SECRET_ACCESS_KEY')
minioUrl =  os.environ.get('MLFLOW_S3_ENDPOINT_URL')
bucketName =  "mlflow"

if  None in (accessID, accessSecret, minioUrl, bucketName):
    print("""
        [!] environment variable is empty! run \'source .env\' to load it from the .env file
        AWS_ACCESS_KEY_ID=
        AWS_SECRET_ACCESS_KEY=
        MLFLOW_S3_ENDPOINT_URL=
        AWS_BUCKET_NAME=
    """ % (accessID, accessSecret, minioUrl, bucketName))
    raise RuntimeError

minioUrlHostWithPort = minioUrl.split('//')[1]
print('[*] minio url: ',minioUrlHostWithPort)

s3Client = Minio(
    endpoint=minioUrlHostWithPort,
    access_key=accessID,
    secret_key=accessSecret,
    secure=False
)

try:
    s3Client.make_bucket(bucketName)
except S3Error as e:
    print(e)

print(f"buckercreated: {bucketName}")


[*] minio url:  s3:9000
S3 operation failed; code: BucketAlreadyOwnedByYou, message: Your previous request to create the named bucket succeeded and you already own it., resource: /mlflow, request_id: 17E2ADC3B6CF796E, host_id: 25d03083-3843-4a00-bb51-f48ae4c1a1ef, bucket_name: mlflow
buckercreated: mlflow


In [3]:
import mlflow
mlflow.set_tracking_uri("http://mlflow_container_ui:8000")

In [8]:
from mlflow.exceptions import RestException

experiment_name = "test_t"

try:
    experiment_id = mlflow.create_experiment(experiment_name, artifact_location="s3://mlflow/data")
    https://10.10.8.9:9000/{bucket}/.../...
    # For remote MinIO
    # hhtp://{host}:{port}/{name_of_bucket}/..
except RestException as e:
    print(e)
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id

print(experiment_id)

16


### структура запуска в MLFlow имеет вид:
```python
import mlflow

mlflow.start_run()
mlflow.log_param("мой параметр", "<значение параметра>")
mlflow.log_metric("оценка", "<значение оценки>")
mlflow.end_run()
```
### вариант запуска через with, который автоматически завершает выполнение в конце блока with.
```python
with mlflow.start_run() as run:
    mlflow.log_param("мой параметр", "<значение параметра>")
    mlflow.log_metric("оценка", "<значение оценки>")
```

### Основные методы логирования в MLFlow
#### MLflow поддерживает логгирование параметров, используемых в экспериментах. Параметры могут иметь любой тип и регистрируются с помощью следующего синтаксиса:
```python
mlflow.log_param("num_epochs", 20)
```

#### MLflow также предоставляет удобный способ ведения журналов для нескольких параметров, указывая их с помощью словаря. Несколько платформ также могут передавать параметры моделям с помощью словарей, поэтому это удобный способ их регистрации в эксперименте.
```python
params = {
    "num_epochs": 20,
    "dropout_rate": .6,
    "objective": "binary_crossentropy"
}

mlflow.log_params(params)
```
#### для логирования метрик используется метод ниже:
```python
mlflow.log_metric("rmse", rmse)
```
#### Регистрация модели scikit-learn в качестве артефакта MLflow для текущего запуска:
```python
mlflow.sklearn.log_model(lr, "model")
```
#### Также можно загружать в артефакты другие какие-либо файлы (графики, и т.д):
```python
mlflow.log_artifact('Picture.png')
```

In [9]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Train the model
model = LogisticRegression(**params)
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)


In [10]:
# Start an MLflow run
# mflow.autolog() --> логгирование всех данных об эксперименте
with mlflow.start_run(run_name="First_run_1", experiment_id=experiment_id):
    # Log the hyperparameters
    mlflow.log_params(params)

    # Log the loss metric
    mlflow.log_metric("accuracy", accuracy)

    # Set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Infer the model signature
    signature = infer_signature(X_train, model.predict(X_train))

    # Log the model
    model_info = mlflow.sklearn.log_model(model, "model")


In [11]:
# Задание метрик модели
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

In [22]:


if __name__ == "__main__":
    mlflow.set_experiment("test_t")
    mlflow.start_run()
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    X, y = datasets.load_iris(return_X_y=True)

    # Split the data into training and test sets
    train_x, test_x, train_y, test_y = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Разделите данные на обучающие и тестовые наборы. (0.75, 0.25).
    # train, test = train_test_split(data)

    # Создание признаков и меток
    # train_x = train.drop(["progression"], axis=1)
    # test_x = test.drop(["progression"], axis=1)
    # train_y = train[["progression"]]
    # test_y = test[["progression"]]

    #изменение параметров для разных запуксков
    alpha = 0.35
    l1_ratio = 0.75

    # Запуск обучения моедли ElasticNet
    lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
    lr.fit(train_x, train_y)
    predicted_qualities = lr.predict(test_x)
    (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

    # Выводим метрику
    print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    print("  RMSE: %s" % rmse)
    print("  MAE: %s" % mae)
    print("  R2: %s" % r2)
    # Логгируем метрики модели в MLFlow
    mlflow.log_param("alpha", alpha)
    mlflow.log_param("l1_ratio", l1_ratio)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("r2", r2)
    mlflow.log_metric("mae", mae)
    mlflow.sklearn.log_model(lr, "model")

    # Вычисление пути регуляризации с использованием ElasticNet
    eps = 5e-3
    print("Computing regularization path using the elastic net.")
    alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio)

    # Вывод графика
    fig = plt.figure(1)
    ax = plt.gca()

    colors = cycle(["b", "r", "g", "c", "k"])
    neg_log_alphas_enet = -np.log10(alphas_enet)
    for coef_e, c in zip(coefs_enet, colors):
        l2 = plt.plot(neg_log_alphas_enet, coef_e, linestyle="--", c=c)

    plt.xlabel("-Log(alpha)")
    plt.ylabel("coefficients")
    title = "ElasticNet Path by alpha for l1_ratio = " + str(l1_ratio)
    plt.title(title)
    plt.axis("tight")

    # Сохранение графика
    fig.savefig("ElasticNet-paths.png")
    plt.close(fig)
    
    # Логирование артефактов (график)
    mlflow.log_artifact("ElasticNet-paths.png")
    # X.to_csv('Dataset.csv')
    mlflow.log_artifact("Dataset.csv")
    mlflow.end_run()

Elasticnet model (alpha=0.350000, l1_ratio=0.750000):
  RMSE: 0.3051592097710266
  MAE: 0.27146284972476026
  R2: 0.8667568696704775
Computing regularization path using the elastic net.


In [21]:
mlflow.end_run()

In [23]:
import mlflow
from mlflow.models import infer_signature

import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Загрузите набор данных iris
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Определите гиперпараметры модели
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}

# Обучите модель
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)

# Прогнозирование на основе тестового набора
y_pred = lr.predict(X_test)

# Вычисляем показатели
accuracy = accuracy_score(y_test, y_pred)
# Set our tracking server uri for logging
# mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

# Создайте новый эксперимент с MLflow
mlflow.set_experiment("test_t")

# Запустите MLflow
with mlflow.start_run(run_name="Register model", experiment_id=experiment_id):
    # Регистрируйте гиперпараметры
    mlflow.log_params(params)

    # Регистрируйте показатель потерь
    mlflow.log_metric("accuracy", accuracy)

    # Установите тег, который мы сможем использовать, чтобы напоминать себе, для чего был этот забег
    mlflow.set_tag("Training Info", "Basic LR model for iris data")

    # Выведите сигнатуру модели
    signature = infer_signature(X_train, lr.predict(X_train))

    # Регистрируйте модель
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

# обозначение входоных и выходных данных а также прогнозирование
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)
predictions = loaded_model.predict(X_test)

iris_feature_names = datasets.load_iris().feature_names

result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions

print(result[:4])

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/07/16 11:35:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 4
Created version '4' of model 'tracking-quickstart'.


   sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                6.1               2.8                4.7               1.2   
1                5.7               3.8                1.7               0.3   
2                7.7               2.6                6.9               2.3   
3                6.0               2.9                4.5               1.5   

   actual_class  predicted_class  
0             1                1  
1             0                0  
2             2                2  
3             1                1  
